In [1]:
import speech_recognition as sr
import moviepy.editor as mp
import subprocess
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
from pydub.silence import split_on_silence
import numpy as np
import sys
import os
from googletrans import Translator
from convert_millis_srt import convert_millis_srt

In [2]:
def extractAudio(input_file, audio_file_name):
    """
    Extracts the audio of a video file into a wav file, for easyness of conversion
    """
    command = "ffmpeg -hide_banner -loglevel warning -i {} -b:a 192k -ac 1 -ar 16000 -vn {}".format(input_file, audio_file_name)

    try:
        ret = subprocess.call(command, shell=True)
        print("Extracted audio to audio/{}".format(audio_file_name.split("/")[-1]))
    except Exception as e:
        print("Error: ", str(e))
        exit(1)

In [3]:
fileaudio = "dassens.wav"

extractAudio("dassens.mp4", fileaudio)

Extracted audio to audio/dassens.wav


File 'dassens.wav' already exists. Overwrite? [y/N] Not overwriting - exiting


In [4]:
#adjust target amplitude
#def match_target_amplitude(sound, target_dBFS):
#    change_in_dBFS = target_dBFS - sound.dBFS
#    return sound.apply_gain(change_in_dBFS)

#Convert wav to audio_segment
belowFS = 12

audio_segment = AudioSegment.from_wav(fileaudio)
nonsilent_data = detect_nonsilent(audio_segment,
                                  min_silence_len=500,
                                  silence_thresh=audio_segment.dBFS-belowFS,
                                  seek_step=1)

#ind = 1
#for chunk in nonsilent_data:
#    print(ind)
#    str_times = convert_millis_srt(chunk[0]) + " --> " + convert_millis_srt(chunk[1])
#    print(str_times)    
#    print("")
#    ind += 1

In [5]:
sound = AudioSegment.from_wav(fileaudio)  

# split audio sound where silence is 700 miliseconds or more and get chunks
chunks = split_on_silence(sound,
                          # experiment with this value for your target audio file
                          min_silence_len = 500,
                          # adjust this per requirement
                          silence_thresh = sound.dBFS-belowFS,
                          # keep the silence for 1 second, adjustable as well
                          keep_silence=500)
    
folder_name = "audio-chunks"
# create a directory to store the audio chunks
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

In [6]:
# Audio language
#language_audio = 'pt-PT'
language_audio = 'nl-NL'

# Destination language
language_subtitles = 'en'

# Speech recognizer instance
recognizer = sr.Recognizer()

# Translator
translator = Translator()

f = open("srts2.srt", "w")

for i, audio_chunk in enumerate(chunks, start=1):
    # export audio chunk and save it in
    # the `folder_name` directory.
    chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
    audio_chunk.export(chunk_filename, format="wav")

    with sr.AudioFile(chunk_filename) as source:
        audio_listened = recognizer.record(source)
        # try converting it to text
        try:
            text = recognizer.recognize_google(audio_listened, language=language_audio)
        except sr.UnknownValueError as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
                
    f.write(str(i))
    f.write("\n")
    str_times = convert_millis_srt(nonsilent_data[i-1][0]) + " --> " + convert_millis_srt(nonsilent_data[i-1][1])
    f.write(str_times)
    f.write("\n")

    if language_audio[:2] == language_subtitles:
        f.write(text)
    else:
        translated = translator.translate(text, dest=language_subtitles, src=language_audio[:2])
        f.write(translated.text)
        
    f.write("\n")
    f.write("")
    f.write("\n")
    
f.close()

Error: 
Error: 
Error: 
Error: 
Error: 
